# Installs

In [1]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 64.5 MB/s eta 0:00:00



This may take a while

In [2]:
!pip install torchsummaryX==1.3.0
!pip install pandas==1.5.2
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 98.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.25.0 requires pandas>=1.5.3, but you have pandas 1.5.2 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.2 which is incompatible.
ibis-framework 9.2.0 requires pandas<3,>=1.5.3, but you have pandas 1.5.2 which is incompatible.
mizani 0.13.0 requires pandas>=2.2.0, but you have pandas 1.5.2 which is incompatible.
plotnine 0.14.0 requires pandas>=2.2.0, but you have pandas 1.5.2 which is incompatible.
xarray 2024.10.0 requires pandas>=2.1, 

# Imports

In [1]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [4]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"gapbu123","key":"9aead84bcd4b2de7b74761101e6ffe3f"}')

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
!kaggle competitions download -c 11-785-hw3p2-f24

100% 3.97G/3.97G [00:24<00:00, 267MB/s]
100% 3.97G/3.97G [00:24<00:00, 175MB/s]


In [6]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f24-hw3p2  ctcdecode  hw3p2asr-f24.zip  sample_data
'''
!unzip -q /content/11-785-hw3p2-f24.zip
!ls

11785-f24-hw3p2  11-785-hw3p2-f24.zip  ctcdecode  sample_data


# Dataset and Dataloader

In [2]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [8]:
import sys
print(sys.version)


3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]


### Train Data

In [3]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    def __init__(self, root, partition="train-clean-100", phonemes=PHONEMES):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.transcript_dir = os.path.join(root, partition, "transcript")

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = phonemes

        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        assert self.length == len(self.transcript_files)

        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        self.mfccs, self.transcripts = [], []

        for i in range(self.length):
            mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[i]))
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / (np.std(mfcc, axis=0) + 1e-8) # Cepstral normalization
            self.mfccs.append(mfcc)

            transcript = np.load(os.path.join(self.transcript_dir, self.transcript_files[i]))[1:-1] # Remove SOS and EOS
            self.transcripts.append([self.PHONEMES.index(p) for p in transcript])

    def __len__(self):

        '''
        What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = torch.FloatTensor(self.mfccs[ind])
        transcript = torch.LongTensor(self.transcripts[ind])

        return mfcc, transcript

    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [item[0] for item in batch]
        # batch of output phonemes
        batch_transcript = [item[1] for item in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = [mfcc.shape[0] for mfcc in batch_mfcc]

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True)
        lengths_transcript = [transcript.shape[0] for transcript in batch_transcript]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.LongTensor(lengths_mfcc), torch.LongTensor(lengths_transcript)



### Test Data

In [4]:
# Test Dataloader
#TODO

class AudioDatasetTest(torch.utils.data.Dataset):

    def __init__(self, root, partition="test-clean"):

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.length = len(self.mfcc_files)

        self.mfccs = []

        for mfcc_name in self.mfcc_files:
            mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_name))
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / (np.std(mfcc, axis=0) + 1e-8)
            self.mfccs.append(mfcc)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        mfcc = torch.FloatTensor(self.mfccs[ind])

        return mfcc

    def collate_fn(self, batch):
        batch_mfcc_pad = pad_sequence(batch, batch_first=True)
        lengths_mfcc = [mfcc.shape[0] for mfcc in batch]

        return batch_mfcc_pad, torch.LongTensor(lengths_mfcc)

### Config - Hyperparameters

In [13]:
root = '/content/11785-f24-hw3p2'

# Feel free to add more items here
config = {
    "beam_width" : 15,
    "lr"         : 0.002,  
    "epochs"     : 25,
    "batch_size" : 64  
}
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [7]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [8]:
# Create objects for the dataset class
train_data = AudioDataset(root, "train-clean-100")
val_data = AudioDataset(root, "dev-clean")
test_data = AudioDatasetTest(root, "test-clean")

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset=train_data,
    num_workers=8,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=True,
    collate_fn=train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset=val_data,
    num_workers=2,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=False,
    collate_fn=val_data.collate_fn
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_data,
    num_workers=2,
    batch_size=config['batch_size'],
    pin_memory=True,
    shuffle=False,
    collate_fn=test_data.collate_fn
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [9]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1653, 28]) torch.Size([64, 217]) torch.Size([64]) torch.Size([64])


# NETWORK

### Pyramid Bi-LSTM (pBLSTM)

In [25]:
torch.cuda.empty_cache()

import torch
import torch.nn as nn

class PermuteBlock(nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

class LockedDropout(nn.Module):
    def __init__(self, dropout=0.25):
        super(LockedDropout, self).__init__()
        self.dropout = dropout

    def forward(self, x):
        if not self.training or self.dropout == 0:
            return x
        mask = x.new_empty(1, x.size(1), x.size(2), requires_grad=False).bernoulli_(1 - self.dropout)
        mask = mask / (1 - self.dropout)
        return mask * x

class pBLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        self.blstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = LockedDropout(0.25)

    def forward(self, x_packed):
        x, x_lens = nn.utils.rnn.pad_packed_sequence(x_packed, batch_first=True)
        x, x_lens = self.trunc_reshape(x, x_lens)
        x = self.dropout(x)
        x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        lstm_out, _ = self.blstm(x_packed)
        return lstm_out

    def trunc_reshape(self, x, x_lens):
        if x.size(1) % 2 != 0:
            x_lens = x_lens - 1
            max_len = x_lens.max().item()
            x = x[:, :max_len, :]
        batch_size, time_steps, feature_dim = x.size()
        x = x.contiguous().view(batch_size, time_steps // 2, feature_dim * 2)
        x_lens = x_lens // 2
        return x, x_lens

class Encoder(nn.Module):
    def __init__(self, input_size, emb_size=1024):
        super(Encoder, self).__init__()
        self.embedding = nn.Conv1d(input_size, emb_size, kernel_size=3, padding=1)
        self.pblstm1 = pBLSTM(input_size=emb_size * 2, hidden_size=emb_size)
        self.pblstm2 = pBLSTM(input_size=emb_size * 4, hidden_size=emb_size)

    def forward(self, x, x_lens):
        x = x.transpose(1, 2)
        x = self.embedding(x)
        x = x.transpose(1, 2)
        x_packed = nn.utils.rnn.pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        x_packed = self.pblstm1(x_packed)
        x_packed = self.pblstm2(x_packed)
        encoder_out, encoder_lens = nn.utils.rnn.pad_packed_sequence(x_packed, batch_first=True)
        return encoder_out, encoder_lens

class Decoder(nn.Module):
    def __init__(self, embed_size=2048, output_size=41): 
        super(Decoder, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(embed_size, 1024, bias=False),
            PermuteBlock(),
            nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            PermuteBlock(),
            nn.GELU(),

            nn.Linear(1024, 512, bias=False),
            PermuteBlock(),
            nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            PermuteBlock(),
            nn.GELU(),

            nn.Linear(512, 256, bias=False),
            PermuteBlock(),
            nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            PermuteBlock(),
            nn.GELU(),

            nn.Linear(256, 128, bias=False),
            PermuteBlock(),
            nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            PermuteBlock(),
            nn.GELU(),

            nn.Linear(128, output_size, bias=True)
        )

        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        out = self.mlp(encoder_out)
        out = self.softmax(out)
        return out


import torchaudio.transforms as tat

class ASRModel(nn.Module):
    def __init__(self, input_size=28, embed_size=2048, output_size=41):
        super(ASRModel, self).__init__()

        self.augmentations = nn.Sequential(
            PermuteBlock(),
            tat.TimeMasking(time_mask_param=65),
            tat.FrequencyMasking(freq_mask_param=5),
            PermuteBlock(),
        )

        self.encoder = Encoder(input_size=input_size, emb_size=embed_size//2)
        self.decoder = Decoder(embed_size=embed_size, output_size=output_size)

    def forward(self, x, lengths_x):
        if self.training:
            x = self.augmentations(x)
        encoder_out, encoder_lens = self.encoder(x, lengths_x)
        decoder_out = self.decoder(encoder_out)
        return decoder_out, encoder_lens



model = ASRModel(
    input_size  = 28,
    embed_size  = 1024,
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)


ASRModel(
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): TimeMasking()
    (2): FrequencyMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): Conv1d(28, 512, kernel_size=(3,), stride=(1,), padding=(1,))
    (pblstm1): pBLSTM(
      (blstm): LSTM(1024, 512, batch_first=True, bidirectional=True)
      (dropout): LockedDropout()
    )
    (pblstm2): pBLSTM(
      (blstm): LSTM(2048, 512, batch_first=True, bidirectional=True)
      (dropout): LockedDropout()
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=False)
      (1): PermuteBlock()
      (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): PermuteBlock()
      (4): GELU(approximate='none')
      (5): Linear(in_features=1024, out_features=512, bias=False)
      (6): PermuteBlock()
      (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): Pe

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations.PermuteBlock_0,-,"[64, 28, 2936]",NaN,NaN
1_augmentations.TimeMasking_1,-,"[64, 28, 2936]",NaN,NaN
2_augmentations.FrequencyMasking_2,-,"[64, 28, 2936]",NaN,NaN
3_augmentations.PermuteBlock_3,-,"[64, 2936, 28]",NaN,NaN
4_encoder.Conv1d_embedding,"[28, 512, 3]","[64, 512, 2936]",43520.0,126271488.0
5_encoder.pblstm1.LockedDropout_dropout,-,"[64, 1468, 1024]",NaN,NaN
6_encoder.pblstm1.LSTM_blstm,-,"[21060, 1024]",6299648.0,6291456.0
7_encoder.pblstm2.LockedDropout_dropout,-,"[64, 734, 2048]",NaN,NaN
8_encoder.pblstm2.LSTM_blstm,-,"[10517, 1024]",10493952.0,10485760.0


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [26]:
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR

criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=1e-5) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=PHONEMES, beam_width=config['beam_width'], blank_id=0, log_probs_input=True)

scheduler = CosineAnnealingLR(optimizer, T_max=config['epochs'])

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

In [27]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from ctcdecode import CTCBeamDecoder  # Ensure you have ctcdecode installed

# CTC Loss 설정
criterion = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)

# AdamW 옵티마이저 설정 (weight decay = 0.01)
optimizer = AdamW(model.parameters(), lr=config['lr'], weight_decay=0.01)

# CTC Beam Decoder 설정
decoder = CTCBeamDecoder(labels=PHONEMES, beam_width=config['beam_width'], blank_id=0, log_probs_input=True)

# Cosine Annealing Scheduler 설정
scheduler = CosineAnnealingLR(optimizer, T_max=config['epochs'])

# Mixed Precision을 위한 GradScaler 설정
scaler = torch.cuda.amp.GradScaler()


# Decode Prediction

In [28]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # Look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(output, seq_lens=output_lens) #lengths - list of lengths

    pred_strings                    = []

    for i in range(output_lens.shape[0]):
        # Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred_indices = beam_results[i][0][:out_lens[i][0]].tolist()
        pred_string = ''.join([PHONEME_MAP[idx] for idx in pred_indices])
        pred_strings.append(pred_string)

    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]
        label_indices = label[i][:label_lens[i]].tolist()
        label_string = ''.join([PHONEME_MAP[idx] for idx in label_indices])
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size

    return dist

# Test Implementation

In [29]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 734, 41])
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(7.7435, device='cuda:0', grad_fn=<MeanBackward0>)
208.015625


# WandB

You will need to fetch your api key from wandb.ai

In [30]:
import wandb
wandb.login(key="f20506dbcca14d82ab5324952dad2c08e99085e2")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [31]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

# Train Functions

In [32]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [33]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric='valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [34]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
# epoch_model_path = # set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = '/content/checkpoints/best_model.pth'

In [ ]:
torch.cuda.empty_cache()
gc.collect()

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = optimizer.param_groups[0]['lr']

    train_loss = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist = validate_model(model, val_loader, decoder, LABELS)
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    # save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    # wandb.save(epoch_model_path)
    # print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/25


Train:   4%|▍         | 18/446 [00:13<05:13,  1.36it/s, loss=4.0551, lr=0.002000]

# Generate Predictions and Submit to Kaggle

In [33]:
model = ASRModel(
    input_size  = 28,
    embed_size  = 192,
    output_size = len(PHONEMES)
).to(device)

model, _, _, _, _ = load_model(best_model_path, model, 'valid_dist', optimizer, scheduler)

In [34]:
# Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 10

test_decoder    = CTCBeamDecoder(labels=PHONEMES, beam_width=TEST_BEAM_WIDTH, blank_id=0, log_probs_input=True)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string= decode_prediction(h, lh, test_decoder, PHONEME_MAP=LABELS)
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing


100%|██████████| 41/41 [00:31<00:00,  1.30it/s]


In [35]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('/content/submission/submission_10.csv', index = False)

In [ ]:
# !kaggle competitions submit -c hw3p2-785-f24 -f submission.csv -m "I made it!"